In [34]:
import pandas as pd
import numpy as np

## 纠正标签

In [35]:
#生成corrected_Combined_News_DJIA.csv
data = pd.read_csv('Combined_News_DJIA.csv')
data['Label'] = data['Label'].shift(-1).fillna(0).astype(int)
data['Label'] = data['Label'].shift(-1).fillna(0).astype(int)
data.to_csv('corrected_Combined_News_DJIA.csv',index=False)



## 合并所有新闻

In [137]:
corr_data = pd.read_csv('corrected_Combined_News_DJIA.csv')

corr_data['all'] = corr_data[corr_data.columns[2:]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)

combin_data = pd.DataFrame()

combin_data['1'] = corr_data['Label']
combin_data['all'] = corr_data['all']

combin_data.to_csv('combinedText_News_DJIA.csv',index=False)

## 划分数据集

In [85]:
combin_data.head()

,Label,all
0,1,"b""Georgia 'downs two Russian warplanes' as cou..."
1,0,b'Why wont America and Nato help us? If they w...
2,0,b'Remember that adorable 9-year-old who sang a...
3,1,b' U.S. refuses Israel weapons to attack Iran:...
4,1,b'All the experts admit that we should legalis...


In [86]:
round(len(combin_data['Label'])*0.9)

1790

In [87]:
#  随机打乱
# combin_data=combin_data.sample(frac=1.0)


In [139]:
Train = combin_data[:round(len(combin_data['1'])*0.9)]
Test = combin_data[round(len(combin_data['1'])*0.9):]

In [111]:
Train.tail()

,Label,all
1785,0,Child poverty in Scotland is now so severe tha...
1786,1,Malcom Turnbull becomes Prime Minister of Aust...
1787,1,Egyptian Billionaire who wants to purchase pri...
1788,0,Tuna and mackerel populations suffer catastrop...
1789,0,"Efficiency up, turnover down: Sweden experimen..."


In [140]:
Train.to_csv('train.csv',index=False)
Test.to_csv('test.csv',index=False)

In [90]:
#最大长度
print(combin_data['all'].map(lambda x: len(x)).max())
#最小长度
print(combin_data['all'].map(lambda x: len(x)).min())

4424
1473


## 去除特殊符号，停用词

In [92]:
# 停止词
from nltk.corpus import stopwords
stop = stopwords.words('english')

# 数字
import re
def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))

# 特殊符号
def isSymbol(inputString):
    return bool(re.match(r'[^\w]', inputString))

# lemma
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def check(word):
    """
    如果需要这个单词，则True
    如果应该去除，则False
    """
#     word= word.lower()
#     if word in stop:
#         return False
#     elif hasNumbers(word) or isSymbol(word):
#         return False
#     else:
#         return True
    if hasNumbers(word) or isSymbol(word):
        return False
    else:
        return True

# 把上面的方法综合起来
def preprocessing(sen):
    res = []
    for word in sen:
        if check(word):
            # 这一段的用处仅仅是去除python里面byte存str时候留下的标识。。之前数据没处理好，其他case里不会有这个情况
            word = word.lower().replace("b'", '').replace('b"', '').replace('"', '').replace("'", '')
            res.append(wordnet_lemmatizer.lemmatize(word))
    return res

In [93]:
from nltk.tokenize import word_tokenize
from gensim.models.word2vec import Word2Vec

In [97]:
corpus = combin_data['all'].values.flatten().astype(str)
corpus = [word_tokenize(x) for x in corpus]
corpus = [preprocessing(x) for x in corpus]

In [99]:
model = Word2Vec(corpus, size=128, window=5, min_count=5, workers=4)
vocab = model.wv.vocab

In [100]:
# 词表长度
len(vocab)

10330

In [101]:
print('语料数：', model.corpus_count)
# print('词表长度：', len(model.wv.vocab))

语料数： 1989


## 生成ID文件

## 处理成表格式

In [75]:
"""
description:
author: 
"""

import random
import sys
import getopt
import os
import re
import csv
from nltk.corpus import stopwords
stop = stopwords.words('english')

csv.field_size_limit(sys.maxsize)

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    for stops in stop:
        string = string.strip().strip('"').replace(stops,'')


    
    string = re.sub(r"[^A-Za-z0-9(),!?\.\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"\.", " \. ", string)
    string = re.sub(r"\"", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

def convert_multi_slots_to_single_slots(slots):
    """
    covert the data which text_data are saved as multi-slots, e.g()
    """
    if len(slots) == 1:
        return slots[0]
    else:
        return ' '.join(slots)


def build_vocab(save_dir, data, min_frq=2):
    """
    build vocab from data, which slot 1 is text data, slot 0 is label
    param:
        1: save_dir: the data dir where the vocabulary dict file saved
        2: data: input train data
        3: min_req : words that appare less than {min_req} times will be replaced as <unk>
    output:
        1: vocabulary_dict(key: word, value: idx)  
    """
    word_count = {}
    with open(data, 'r') as csvfile:
        lines = csv.reader(csvfile)
        for items in lines:
            text_data = convert_multi_slots_to_single_slots(items[1:])
            text_data = clean_str(text_data)
            words = set(text_data.split(' '))# word only counts once in a doucment
            for word in words:
                if word: #remove ' '
                    word_count[word] = word_count.get(word, 0) + 1

    # remove unfrequence words
    word_count_filterd = {k:v for k,v in word_count.items() if v >= min_frq}

    words_sorted_by_count = sorted(word_count_filterd.items(), key = lambda x:-x[1])

    vocab_dict = {"<pad>": "0", "<unk>" : '1'}
    with open(save_dir + '/unigram.id', 'w') as vocab_file:
        print("<pad>\t0",file=vocab_file)
        print("<unk>\t1",file=vocab_file)
        idx = 2
        for k, v in words_sorted_by_count:
            print(k + '\t' + str(idx),file=vocab_file)
            vocab_dict[k] = str(idx)
            idx += 1
    return vocab_dict


def load_vocab(v):
    """
    load vocabulary whcih is builded previously
    """
    res = {}
    for line in open(v):
        items = line.strip().split('\t')
        res[items[0]] = items[1]
    return res


def token_lize(vocab, data):
    """
    convert word to corresponding vocabulary id
    """
    with open(data + '.id', "w") as out_f:
        with open(data, 'r') as csvfile:
            lines = csv.reader(csvfile)
            for items in lines:
                text_data = convert_multi_slots_to_single_slots(items[1:])
                text_data = clean_str(text_data)
                words = text_data.split(' ')
                word_ids = []
                for word in words:
                    word_ids.append(vocab.get(word, '1')) #1 == <unk>
                text_data_ids = ' '.join(word_ids)
                lable = str(int(items[0])) # add -1 bias to ensure min label value is 0
                print(';'.join([lable, text_data_ids]),file=out_f)


def split_train_dev(data, dev_dir, rate):
    """
    split train data to dev_train and dev sets
    """
    with open(data, 'r') as f:
        lines = f.readlines()
        random.shuffle(lines)
        dev_idx = int(len(lines) * rate)

    train_file = dev_dir + '/dev_train.csv'
    dev_file = dev_dir + '/dev.csv'

    with open(dev_file, 'w') as f:
        f.write(''.join(lines[0:dev_idx]))

    with open(train_file, 'w') as f:
        f.write(''.join(lines[dev_idx:]))

    return train_file, dev_file

def process(train_file, test_file, frequence_cut, save_dir, dev_file=''):

    """
    build vocabulary dict from train_file
    tokenlize all datas
    """
    if not os.path.exists(v):
        print('vocab not exist, build from data',file=sys.stderr)
        vocab = build_vocab(save_dir, train_file, frequence_cut)
    else:
        vocab = load_vocab(v)
    for _file in [train_file, test_file]:
        token_lize(vocab, _file)
    if dev_file:
        token_lize(vocab, dev_file)




In [76]:
v = ''
#需要改写
data_dir = 'DJIA'
dev_rate = 0.1
frequence_cut = 2 #remove words that only apear in one document

dev_dir = data_dir.strip('/') + '_for_dev'
os.system('mkdir -p %s' % dev_dir)
os.system('cp %s/test.csv %s/dev_test.csv' % (data_dir, dev_dir))

dev_train_file, dev_file = split_train_dev(data_dir + '/train.csv', dev_dir, dev_rate)
dev_test_file = dev_dir + '/dev_test.csv'

test_file = data_dir + '/test.csv'
train_file = data_dir + '/train.csv'

#generate data for parameter tuning, default using 10% training data for dev
# process(dev_train_file, dev_test_file, frequence_cut, dev_dir, dev_file)

#use the best parameter and 100% training data to get the model's performance
process(train_file, test_file, frequence_cut, data_dir)




vocab not exist, build from data


In [77]:
len(vocab)

21785

In [78]:
import pandas as pd

In [80]:
train_id=pd.read_csv('DJIA/train.csv.id')

In [81]:
train_id.head()

,1;1
0,1;15 1208 3059 10 53 1 87 1452 43 3 3622 4679 ...
1,0;962 10 388 2 2 216 21 73 347 10 216 21 10 3 ...
2,0;2644 2827 375 17 9 33 571 611 73 140 718 3 4...
3,1;224 21 4 31 4 349 32 8 72 103 565 3 15 23 40...
4,1;403 471 191 1770 151 38 3 3195 2 88 2156 306...


In [82]:

print(train_id['1;1'].map(lambda x: len(x)).max())
print(train_id['1;1'].map(lambda x: len(x)).min())

2121
806


In [83]:
test_id=pd.read_csv('DJIA/test.csv.id')
print(test_id['1;1'].map(lambda x: len(x)).max())
print(test_id['1;1'].map(lambda x: len(x)).min())

1783
946


In [52]:
ag_train_id=pd.read_csv('ag/train.csv.id')
ag_train_id.head()

,2;443 497 2 2051 13509 107 64 3 1009 12 32 13 32 719 8194 4 443 363 27 9441 3050 7 5582 24178 4 42 3733 890 306 2
0,2;20278 1003 837 1225 4116 12 32 13 32 900 816...
1,2;99 9 390 4498 26159 808 12 32 13 32 2214 537...
2,2;100 6765 99 1991 28 820 532 3210 12 32 13 32...
3,2;99 116 4079 5 104 82 152 4 7047 24 11409 5 3...
4,2;132 138 39 4 43 335 44 2494 12 32 13 32 132 ...


In [63]:
print(ag_train_id['2;443 497 2 2051 13509 107 64 3 1009 12 32 13 32 719 8194 4 443 363 27 9441 3050 7 5582 24178 4 42 3733 890 306 2'].map(lambda x: len(x)).max())
print(ag_train_id['2;443 497 2 2051 13509 107 64 3 1009 12 32 13 32 719 8194 4 443 363 27 9441 3050 7 5582 24178 4 42 3733 890 306 2'].map(lambda x: len(x)).min())


801
52
